# Ant Colony Optimization 

- The Ant Colony Optimization Algorithm is a conbinatorial optimization Algorithm
- Inspired by the social behavior of Ant Colonies
- The Social interaction of Ants allow them to solve complex tasks 

![alt text](images/Cover.jpg "Title")


# Simple Ant Colony Optimization (S-ACO)
- Simulates ant behavior to find the shortest path in a graph
- Selects a path with probability based on how many ants visted the route
- After traversing a path leaves a trail called pheromone
- pheromone is the basis of path selection

### Given the Graph

- Find the Shortest path from point 0 to point 8

<img src="images/extendedDB.png" width="400"/>

- The Graph matrix is given below

In [24]:
# Exteded double bridge experiment from Marco Dorigo ACO Book 
# Shortest path is [ 0,  9, 10, 12, 13,  8] or [ 0,  9, 14, 17, 13,  8]

EDBM = [[0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
        [1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0],
        [1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0],
        [0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0],
        [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1],
        [0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0]]

PHM = [[1 for i in range(len(EDBM))] for i in range(len(EDBM))] 

# 1. Identifying Connections in the graph

- To know the connections of a node, use the cost graph
- getting the connections of node 0, get the first row or column of the matrix
- e.g costGraph[0] gets all connections for node 0
- Note: A connection is indicated by a 1

In [26]:
# Connections to a node. Enter a node to get connections 

costGraph = EDBM[:]
pheromoneGraph = PHM[:]

Node = int(input("Enter a Node Number to see its links "))
print("Connected to Node ", Node , " >> ", costGraph[Node])
Connections = [i for i,j in enumerate(costGraph[Node]) if j>0]
print("Node ", Node, "is connected to Nodes >>  " , Connections)

Enter a Node Number to see its links 6
Connected to Node  6  >>  [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Node  6 is connected to Nodes >>   [5, 7]


# 2. Where next should I Making a move

- Assume we are in `Node 0` (you can enter Node zero above to see it's conections)
- The next move we can make can only be to the nodes Node 0 is connected to i.e `1 or 9`
- We select the next Node (hob) by probability $P_{i,j}$ based on the pheromone 
- $P_{i,j}$ is the probability of moving from `i to j`. In our case `0 to 1` or `0 to 9`
- $P_{i,j} = \frac{ \tau_{i,j}^\alpha} {\sum_{l \in N_{i}} \tau_{i,j}^\alpha } $
- where $N_{i}$ is the set of avaialbale choices
- In our example for `Node 0`, $N_{i}$ is `1 and 9`

In [27]:
pheromoneIJS = [ pheromoneGraph[Node][i] for i in Connections]
print(pheromoneIJS)
ProbabilityIJS = [ i/sum(pheromoneIJS) for i in pheromoneIJS]
print(ProbabilityIJS)

[1, 1]
[0.5, 0.5]


# 3. Making a probabilistic move with roulette probability 
- The next node is choosen with a probability by rolling a roullette wheel 
- Let's calculate the roulette wheel probability for each node

<img src="images/roulette.png" width="400"/>

- Then we'll make a next move based on that probability 

In [28]:
# Calculate cumulative sum for roulette wheel 
import random

RouletteProb = [ sum(ProbabilityIJS[:i+1]) for i in range(len(ProbabilityIJS)) ]
print(RouletteProb)
print(Connections)

[0.5, 1.0]
[5, 7]


# Choose next hob based on roulette probability 

In [29]:
# pair connections with probability
# D = dict(zip(Connections, RouletteProb))
# D = tuple(zip(Connections, RouletteProb))

def NextNode(Connections, RouletteProb):
    r = random.random()
    for i in range(len(Connections)):
        if i == 0 and r <= RouletteProb[i]:
            return Connections[i]
        elif r > RouletteProb[i] and r <= RouletteProb[i+1]:
            return Connections[i+1]
        
NextNode(Connections, RouletteProb)

7

# 4. Making Sequential Probabilistic Choices
- Now that we can make one move lets keep moving till we reach our Destination

In [30]:
def Travel(costGraph, pheromoneGraph, source, destination):
    route = [source]
    while source != destination:  
        Connections = [i for i,j in enumerate(costGraph[source]) if j>0]
        pheromoneIJS = [ pheromoneGraph[source][i] for i in Connections]
        ProbabilityIJS = [ i/sum(pheromoneIJS) for i in pheromoneIJS]
        RouletteProb = [ sum(ProbabilityIJS[:i+1]) for i in range(len(ProbabilityIJS)) ]
        #print(Connections, " Connecton to probability >> ", RouletteProb)
        #input("press enter to continue")
        source = NextNode(Connections, RouletteProb)
        route.append(source)
        #print(route)
    return route

# Travel(costGraph, pheromoneGraph, 0, 8)

In [31]:
takeoff = int(input("Please enter your takeoff Node "))
destination = int(input("Please enter your destination Node "))
print(" We are looking for the route from ", takeoff, " To ", destination)

Travel(costGraph, pheromoneGraph, takeoff, destination)

Please enter your takeoff Node 3
Please enter your destination Node 5
 We are looking for the route from  3  To  5


[3, 4, 5]

# 5. Deloop 

- We can observe our route has loops
- We need to eliminate the loops 
- We do this to get a good route with no loops or cycles 

## Delooping Technique
- To eliminate loops we scan from the first node 
- Select Node 0, scan from the end Node backwards to see if you find another Node 0
- If its found, eliminate all nodes from Node 1 to the second occurance of Node 0

<img src="images/deloop.png" width="500"/>

In [33]:
# Eliminate Loops 

def Deloop(route):
    l = len(route)
    for i in range(l):
        for j in range(i+1,l):
            j = l+i-j
            if j < len(route) and route[i] == route[j]:          # list changes as loop is removed 
                # print("Delooping:  ", x)
                [route.remove(k) for k in route[i+1:j+1]]
                # print("Delooped to: ", x)
    return route

Route = Travel(costGraph, pheromoneGraph, takeoff, destination)
Deloop(Route)

[3, 4, 5]

# 6. After Delooping let's add pheromone to the new route

- Pheromone is added on the arcs
- the amount of pheromone deposited is related to the cost of travel
- The amount of pheromone added can either be a constant or a measure of the path cost

In [39]:
def Updatepheromone(route, pheromoneGraph):
    """route : route taken
    pheromoneGraph: pheromone matrix"""
    
    tau = 1/len(route)
    #tau = 1
    for i in range(len(route)-1):
        pheromoneGraph[route[i]][route[i+1]] += tau
        pheromoneGraph[route[i+1]][route[i]] += tau

    return pheromoneGraph

Updatepheromone(Route, PHM)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 2.6666666666666665, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1,
  1,
  1,
  2.6666666666666665,
  1,
  2.6666666666666665,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 [1, 1, 1, 1, 2.6666666666666665, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

# 7. Finally Iterative Construction of Paths Using one Ant 

- Given a graph 
- Find the shortest path

In [48]:
# define parameters

iterations = 10
takeoff = 0
destination = 8

i = 0
while i < iterations:
    LoopRoute = Travel(costGraph, pheromoneGraph, takeoff, destination)
    Route = Deloop(LoopRoute)
    PHM = Updatepheromone(Route, PHM)
    i += 1
    print(Route)

[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 14, 17, 13, 8]
[0, 9, 10, 12, 13, 8]
[0, 9, 10, 12, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 14, 18, 17, 13, 8]


# Summary

# 1. Given a Node, 

- Get it's Connections 
- Get the pheromone intensities from each connection
- Calculate the trasition probability of each Node
- For computational Purposes, compute the Rouleete probability from the correspondind transition probability

In [50]:
Node = 10
Connections = [i for i,j in enumerate(costGraph[Node]) if j>0]
pheromoneIJS = [ pheromoneGraph[Node][i] for i in Connections]
ProbabilityIJS = [ i/sum(pheromoneIJS) for i in pheromoneIJS]
RouletteProb = [ sum(ProbabilityIJS[:i+1]) for i in range(len(ProbabilityIJS)) ]

# 2. Choose Next Move 

- From the connections and Roulette probability 
- choose the next move using the Roulette probability 

In [51]:
def NextNode(Connections, RouletteProb):
    
    """ takes a list of connections and corresponding Roulette probabilities
    and returns an one connetion from the list of connetions 
    
    connections: List of nodes
    RouletteProb: List corresponding node roulette probability
    returns: int Node choosen """
    
    r = random.random()
    for i in range(len(Connections)):
        if i == 0 and r <= RouletteProb[i]:
            return Connections[i]
        elif r > RouletteProb[i] and r <= RouletteProb[i+1]:
            return Connections[i+1]
        
NextNode(Connections, RouletteProb)

12

# 3. Make a Tour
- Make sequencial probabilistic choices from a takeoff Node to a destination Node
- given a connection graph (costGraph), pheromone graph (pheromoneGraph), takeoff point (source) and destination, make a tour

In [52]:
def Travel(costGraph, pheromoneGraph, source, destination):
    """ takes a 
    - connection graph (costGraph) a square matrix, 
    - pheromone graph (pheromoneGraph) a square matrix same size as costGraph, 
    - takeoff point (source) and destination are the int Node number
    
    Returns a complete tour as a List
    """
    
    route = [source]
    while source != destination:  
        Connections = [i for i,j in enumerate(costGraph[source]) if j>0]
        pheromoneIJS = [ pheromoneGraph[source][i] for i in Connections]
        ProbabilityIJS = [ i/sum(pheromoneIJS) for i in pheromoneIJS]
        RouletteProb = [ sum(ProbabilityIJS[:i+1]) for i in range(len(ProbabilityIJS)) ]
        source = NextNode(Connections, RouletteProb)
        route.append(source)
    return route

# 4. Remove Loops from Tour

- Avoid reinforcing Loops
- Use Scanning Technique to remove Loops

In [53]:
# Eliminate Loops 
def Deloop(route):
    l = len(route)
    for i in range(l):
        for j in range(i+1,l):
            j = l+i-j
            if j < len(route) and route[i] == route[j]:          # list changes as loop is removed 
                # print("Delooping:  ", x)
                [route.remove(k) for k in route[i+1:j+1]]
                # print("Delooped to: ", x)
    return route

Deloop(Route)

[0, 9, 14, 18, 17, 13, 8]

# 5. Update Pheromone

- Add Pheromone to the visited arcs
- the amount added deoends on the lenght of the tour
- the longer the tour the less pheromone is added
- pheromone added is 1 divided by length of tour 

In [54]:
def Updatepheromone(route, pheromoneGraph):
    """ takes a route and use it to add pheromone to arcs in the 
    route
    
    route : route taken
    pheromoneGraph: pheromone matrix
    
    returns pheromone graph """
    
    tau = 1/len(route)
    #tau = 1
    for i in range(len(route)-1):
        pheromoneGraph[route[i]][route[i+1]] += tau
        pheromoneGraph[route[i+1]][route[i]] += tau

    return pheromoneGraph

# 6. Conclusion 

- Given a Graph of Connections, make a tour
- design a pheromone matrix
- Make a tour and update the pheromone matrix 

In [55]:

def Ant(costGraph, takeoff, destination, iterations):
    """ takes a costgraph, takeoff point, destination point and number of times to travel (iterations)
    and return a tour """
    PRM = [[1 for i in range(len(costGraph))] for j in range(len(costGraph))]
    while iterations > 0:
        LoopRoute = Travel(costGraph, PRM, takeoff, destination)
        Route = Deloop(LoopRoute)
        PHM = Updatepheromone(Route, PRM)
        iterations -= 1
        print(Route)
    return {"Route": Route, "PRM":PRM}

# Bonus

### Play With your Ant

In [57]:
iterations = 110
takeoff = 0
destination = 8

Ant(EDBM, takeoff, destination, iterations)

[0, 9, 10, 15, 14, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 14, 15, 10, 12, 13, 8]
[0, 9, 14, 17, 13, 8]
[0, 9, 14, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 10, 15, 16, 17, 13, 8]
[0, 9, 10, 15, 16, 17, 13, 8]
[0, 9, 14, 17, 16, 12, 13, 8]
[0, 9, 14, 15, 16, 17, 13, 8]
[0, 9, 14, 17, 16, 12, 13, 8]
[0, 9, 14, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 10, 12, 16, 17, 13, 8]
[0, 9, 14, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 14, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 14, 17, 16, 12, 13, 8]
[0, 9, 10, 12, 16, 17, 13, 8]
[0, 9, 10, 12, 16, 17, 13, 8]
[0, 9, 14, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 14, 17, 16, 12, 13, 8]
[0, 9, 14, 18, 17, 13, 8]
[0, 9, 14, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 14, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 10, 12, 13, 8]
[0, 9, 10, 12, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 14, 17, 13, 8]
[0, 9, 14, 17, 13, 8]
[0, 9, 14, 17,

{'PRM': [[1,
   3.777777777777779,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   13.664682539682525,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  [3.777777777777779,
   1,
   3.777777777777779,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  [1,
   3.777777777777779,
   1,
   3.777777777777779,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  [1,
   1,
   3.777777777777779,
   1,
   3.777777777777779,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  [1,
   1,
   1,
   3.777777777777779,
   1,
   3.777777777777779,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  [1,
   1,
   1,
   1,
   3.777777777777779,
   1,
   3.777777777777779,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  [1,
   1,
   1,
   1,
   1,
   3.777777777777779,
   1,
   3.777777777777779,
   1,
   1,
   1,
   1,
   1,


# More Elegant

In [58]:
def NextNode(Connections, RouletteProb):
    
    """ takes a list of connections and corresponding Roulette probabilities
    and returns an one connetion from the list of connetions 
    
    connections: List of nodes
    RouletteProb: List corresponding node roulette probability
    returns: int Node choosen """
    
    r = random.random()
    for i in range(len(Connections)):
        if i == 0 and r <= RouletteProb[i]:
            return Connections[i]
        elif r > RouletteProb[i] and r <= RouletteProb[i+1]:
            return Connections[i+1]
        
# Eliminate Loops 
def Deloop(route):
    l = len(route)
    for i in range(l):
        for j in range(i+1,l):
            j = l+i-j
            if j < len(route) and route[i] == route[j]:          # list changes as loop is removed 
                # print("Delooping:  ", x)
                [route.remove(k) for k in route[i+1:j+1]]
                # print("Delooped to: ", x)
    return route

def Travel(costGraph, pheromoneGraph, source, destination):
    """ takes a 
    - connection graph (costGraph) a square matrix, 
    - pheromone graph (pheromoneGraph) a square matrix same size as costGraph, 
    - takeoff point (source) and destination are the int Node number
    
    Returns a complete tour as a List
    """
    
    route = [source]
    while source != destination:  
        Connections = [i for i,j in enumerate(costGraph[source]) if j>0]
        pheromoneIJS = [ pheromoneGraph[source][i] for i in Connections]
        ProbabilityIJS = [ i/sum(pheromoneIJS) for i in pheromoneIJS]
        RouletteProb = [ sum(ProbabilityIJS[:i+1]) for i in range(len(ProbabilityIJS)) ]
        source = NextNode(Connections, RouletteProb)
        route.append(source)
    
    return Deloop(route)

def Updatepheromone(route, pheromoneGraph):
    """ takes a route and use it to add pheromone to arcs in the 
    route
    
    route : route taken
    pheromoneGraph: pheromone matrix
    
    returns pheromone graph """
    
    tau = 1/len(route)
    #tau = 1
    for i in range(len(route)-1):
        pheromoneGraph[route[i]][route[i+1]] += tau
        pheromoneGraph[route[i+1]][route[i]] += tau

    return pheromoneGraph

def Ant(costGraph, takeoff, destination, iterations):
    
    """ takes a costgraph, takeoff point, destination point and number of times to travel (iterations)
    and return a tour """
    PRM = [[1 for i in range(len(costGraph))] for j in range(len(costGraph))]
    while iterations > 0:
        Route = Travel(costGraph, PRM, takeoff, destination)
        PHM = Updatepheromone(Route, PRM)
        iterations -= 1
        print(Route)
    return {"Route": Route, "PRM":PRM}

In [59]:
iterations = 110
takeoff = 0
destination = 8

Tour = Ant(EDBM, takeoff, destination, iterations)

Tour["Route"]

[0, 9, 10, 12, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 10, 11, 12, 13, 8]
[0, 9, 14, 17, 13, 8]
[0, 9, 10, 15, 14, 18, 17, 13, 8]
[0, 9, 10, 15, 16, 17, 13, 8]
[0, 9, 14, 15, 16, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 10, 11, 12, 16, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 14, 15, 16, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 10, 15, 16, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 10, 15, 14, 17, 13, 8]
[0, 9, 10, 12, 13, 8]
[0, 9, 10, 12, 13, 8]
[0, 9, 10, 12, 16, 17, 13, 8]
[0, 9, 10, 15, 16, 17, 13, 8]
[0, 9, 14, 17, 13, 8]
[0, 9, 10, 11, 12, 13, 8]
[0, 9, 10, 15, 16, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 10, 12, 13, 8]
[0, 9, 10, 11, 12, 13, 8]
[0, 9, 14, 15, 16, 12, 13, 8]
[0, 9, 14, 17, 13, 8]
[0, 9, 10, 12, 13, 8]
[0, 9, 14, 17, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 10, 15, 14, 17, 13, 8]
[0, 9, 14, 18, 17, 13, 8]
[0, 9, 10, 11, 12, 13, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 9, 10, 15, 16, 12, 13, 8]
[0, 9, 10, 12, 13, 8]


[0, 9, 14, 18, 17, 13, 8]